# Real Estate Stock Analysis
Intro

In [10]:
#Import the required libraries and dependencies
#CODE HERE

## API to pull stock information into DF - Polygon

In [11]:
#Make an API call to access the current prices for BEKE, OPEN, RDFN, Z, EXPI, AMT, CBRE, WY, ESS, AVB, ARE

## Data cleaning/slicing

In [12]:
#CODE HERE

## Analysis
hvplot and other interactive visualizations

In [13]:
#Calculate daily returns

In [14]:
#Plot daily returns

In [15]:
#Standard deviations

In [16]:
#Cumulative Returns

In [17]:
#Sharpe ratios

In [18]:
#Betas

In [19]:
#Monte Carlo Sim

## Trend Analysis and Investment Recommendations

In [ ]:
#ANSWER HERE